# calculate other_changes

In [53]:
import pandas as pd

# Load your data
df = pd.read_excel("output_cash_flow/all_schools_combined.xlsx")

# Clean column names
df.columns = df.columns.str.strip()

# Convert all columns to numeric where possible, coercing errors to NaN
df = df.apply(pd.to_numeric, errors="coerce")


In [ ]:
# Compute other_changes_in_investment_activities
def compute_other_changes(row):
    if pd.notna(row["net_cash_from_investment_activities"]) and pd.notna(row["capital_expenses"]):
        return row["net_cash_from_investment_activities"] + row["capital_expenses"]
    else:
        return None

df["other_changes_in_investment_activities"] = df.apply(compute_other_changes, axis=1)

$$
\begin{aligned}
\text{other\_changes\_in\_investment\_activities} =\ & 
\text{net\_cash\_from\_investment\_activities} \\
& +\ \text{capital\_expenses}
\end{aligned}
$$

In [ ]:
#other_changes_in_operating_activities
def compute_operating_changes(row):
    if pd.notna(row["net_cash_from_operating_activities"]) and \
       pd.notna(row["total_change_in_net_assets"]) and \
       pd.notna(row["total_non_cash_exp"]) and \
       pd.notna(row["change_in_working_capital"]):
        return row["net_cash_from_operating_activities"] - (
            row["total_change_in_net_assets"] +
            row["total_non_cash_exp"] +
            row["change_in_working_capital"]
        )
    else:
        return None  # Leave cell empty if any input is missing

# Apply the function to compute the new column
df["other_changes_in_operating_activities"] = df.apply(compute_operating_changes, axis=1)

$$
\begin{aligned}
\text{other\_changes\_in\_operating\_activities} =\ & 
\text{net\_cash\_from\_operating\_activities} \\
& - \Big( 
\text{total\_change\_in\_net\_assets} 
+ \text{total\_non\_cash\_exp} 
+ \text{change\_in\_working\_capital}
\Big)
\end{aligned}
$$


In [ ]:
# Compute change_in_long_term_debt 
def compute_change_in_long_term_debt(row):
    if pd.notna(row["long_term_debt_net_proceeds"]) and pd.notna(row["long_term_debt_principal_payments"]):
        return row["long_term_debt_net_proceeds"] - row["long_term_debt_principal_payments"]
    else:
        return None

df["change_in_long_term_debt"] = df.apply(compute_change_in_long_term_debt, axis=1)


$$
\begin{aligned}
\text{change\_in\_long\_term\_debt} =\ & 
\text{long\_term\_debt\_net\_proceeds} \\
& -\ \text{long\_term\_debt\_principal\_payments}
\end{aligned}
$$

In [ ]:
# Compute other_changes_in_financing_activities using the given formula
def compute_other_changes_in_financing(row):
    if pd.notna(row["net_cash_from_financing_activities"]) and \
       pd.notna(row["long_term_debt_net_proceeds"]) and \
       pd.notna(row["long_term_debt_principal_payments"]) and \
       pd.notna(row["cash_flows_from_noncapital_financing_activities"]) and \
       pd.notna(row["cash_flows_from_capital_and_related_financing_activities"]):
        
        debt_change = row["long_term_debt_net_proceeds"] - row["long_term_debt_principal_payments"]
        financing_sum = (
            debt_change +
            row["cash_flows_from_noncapital_financing_activities"] +
            row["cash_flows_from_capital_and_related_financing_activities"]
        )
        return row["net_cash_from_financing_activities"] - financing_sum
    else:
        return None

df["other_changes_in_financing_activities"] = df.apply(compute_other_changes_in_financing, axis=1)


$$
\begin{aligned}
\text{other\_changes\_in\_financing\_activities} =\ & 
\text{net\_cash\_from\_financing\_activities} \\
& - \Big( 
\text{long\_term\_debt\_net\_proceeds} 
- \text{long\_term\_debt\_principal\_payments} \\
& \quad + \text{cash\_flows\_from\_noncapital\_financing\_activities} \\
& \quad + \text{cash\_flows\_from\_capital\_and\_related\_financing\_activities} 
\Big)
\end{aligned}
$$

In [58]:
# Save results
df.to_excel("output_cash_flow/all_schools_combined_filled.xlsx", index=False)
